# Week 2: Supervised Techniques for Regression
## Boston Housing Assignment

 Scikit Learn documentation for this assignment:
 http://scikit-learn.org/stable/modules/model_evaluation.html 
 http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
 http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
 http://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html
 http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html
 http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html
 http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html
 http://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html

 
 Textbook reference materials:
 Geron, A. 2017. Hands-On Machine Learning with Scikit-Learn
 and TensorFlow. Sebastopal, Calif.: O'Reilly. Chapter 3 Training Models
 has sections covering linear regression, polynomial regression,
 and regularized linear models. Sample code from the book is 
 available on GitHub at https://github.com/ageron/handson-ml


## Requirements for this assignment

    1) Use all explanatory variables (with the exception of neighborhood) and all 506 census tract observations from the Boston Housing Study.
    2) Use one of two response variables: (1) the median value of homes in thousands of 1970 dollars or (2) the log median value of homes in thousands of 1970 dollars. 
    3) Employ at least two regression modeling methods selected from those discussed in Chapter 4 of the Géron (2017) textbook: linear regression, ridge regression, lasso regression, and elastic net.
    4) Evaluate these methods within a cross-validation design, using root mean-squared error (RMSE) as an index of prediction error. 


Python scikit-learn should be your primary environment for conducting this research. Note that it is not necessary to employ polynomial regression in this assignment.

## Management problem:

Imagine that you are advising a real estate brokerage firm in its attempt to employ machine learning methods. The firm wants to use machine learning to complement conventional methods for assessing the market value of residential real estate. Of the modeling methods examined in your study, which would you recommend to management, and why?

# Discussion

After getting our data and libraries loaded in section A. Section B is dedicated to performing some exploratory data analysis. Some of data are transformed here to adjust the shape of the values relative to our dependant variable (log_MV). There were a few things I was aiming to do with this assignment the first was to evaluate the. The first was to imagine myself as a scientist with tranditional statistical approaches; EDA followed by selecting the most significant features, and contrast that to a modern machine learning route where the more impactful features are identified. The second was to evaluate the statistics generated by a few different methodologis (linear regression versus ElasticNet and linear regression versus lasso).

As for the first objective (section C) a subset of the columns were selected via exploratory analysis. Linear regression and elastic net are evaluated on the subset data. It is evident from the results below that these two methodologies produce very similar results in terms of coefficient estimates and term significance. To contrast this historical approach (section D), lasso regression was employed to identify the features which were most impactful. After the important features were identified, linear regression was used to contrast the results obtained from the lasso methodology. Again the results obtained were very similar interms of coefficients, error, and feature significance. Likewise as an additonal example a linear regression of all availble terms was conducted demonstrating the presence of overfitting of the model evidenced by the low significance of a number of terms. 

A key learning from this exercise is that machine learning outperforms more traditional approaches. For example when a scientist selects hypothesized impactful features they may have inherent bias because they lack the ability to see the data in the appropriate multidimensional space. Evaluating the data in a tranditional sense results in a model with a mean 8 fold CV RMSE score of ~0.22 in the scale of our dependant varaible. Conversely the most impactful features identified via lasso and subsequently evaluated with linear regression have an 8 fold CV RMSE score of ~0.20 allowing for a small reduction in the RMSE associated with the dependant variable improving our certainty and potentially allowing for larger profits with real estate investment by the firm. An additional learning is the power of lasso to downselect features for linear regression as the regularization methodology reduces the impact of the coefficient for a given feature to essentially zero. As such I would recommend this approach.

Within each regression section (with the exception of linear regression performed on all features; section D2) the following generally applies:
    


    1) The train and test data are split from the data of interest
    2) The model is fit on the training set using a pipeline
    3) Predictions are made on the traning set and a parity plot is generated of predictions vs observed values
    4) The training set is then passed to cross validation and RMSE is reported of each fold along with the average of all folds
    5) Addiitonal Summary statistics for the coefficients and overal model are generated - Training Statistics
    6) The model for the test set is fit using the same pipeline employed for the training set
    7) Predictions are made on the test set and a parity plot is generated of predictions vs observed values
    8) Addiitonal summary statistics for the coefficients and overal model are generated - Test Statistics


# A: Libraries and data read-in

In [ ]:
RANDOM_SEED = 42

# although we standardize X and y variables on input,
# we will fit the intercept term in the models
# Expect fitted values to be close to zero
SET_FIT_INTERCEPT = True

# import base packages into the namespace for this program
import numpy as np
import pandas as pd
from math import sqrt  # for root mean-squared error calculation

#diplay and plotting
import seaborn as sns
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm

#import from SKlearn
import sklearn.linear_model 
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score  
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
import sklearn.metrics as metrics
from sklearn.pipeline import Pipeline

from scipy import stats

# read data for the Boston Housing Study
boston_input = pd.read_csv("C:/Users/bblank/Documents/Northwestern MSDS/datasets/boston.csv")

### Functions to yield information about the models

In [ ]:
def regression_results(y_true, y_pred):

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
   # mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    print('explained_variance: ', round(explained_variance,4))    
   # print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))

 
#This function print's statistics in a nice table format for linear regression
#requires the use of dataframe as inputs for all except the coefs arg.
def model_statistics(x_actual, y_actual, y_predict, coefs):

    matX = pd.DataFrame({"Constant":np.ones(len(x_actual))}).join(pd.DataFrame(x_actual.reset_index(drop=True)))
    MSE = (sum((y_actual-y_predict)**2))/(len(matX)-len(matX.columns))
    

    var_b = MSE*(np.linalg.inv(np.dot(matX.T,matX)).diagonal())
    
    sd_b = np.sqrt(var_b)
    ts_b = coefs/ sd_b

    p_values =[2*(1-stats.t.cdf(np.abs(i),(len(matX)-len(matX.columns)-1))) for i in ts_b]
    

    sd_b = np.round(sd_b,3)
    ts_b = np.round(ts_b,3)
    p_values = np.round(p_values,3)
    coefs = np.round(coefs,4)

    SummaryDF = pd.DataFrame()
    SummaryDF["Coefficients"],SummaryDF["Standard Errors"],SummaryDF["t values"],SummaryDF["Probabilities"] = [coefs,sd_b,ts_b,p_values]

    return(SummaryDF)

# This is a function which returns the fomula of the model with it's coefficients
def pretty_print_coefs(coefs, names = None, sort = False):
    if names == None:
        names = ["X%s" % x for x in range(len(coefs))]
        lst = zip(coefs, names)
    if sort:
        lst = sorted(lst,  key = lambda x:-np.abs(x[0]))
    return " + ".join("%s * %s" % (round(coef, 3), name) 
        for coef, name in lst)


# B: Descriptive Stats and Dataframe inspect

In [ ]:
# check the pandas DataFrame object boston_input
print('\nboston DataFrame (first and last five rows):')
display(boston_input.head())
display(boston_input.tail())

print('\n\nGeneral description of the boston_input DataFrame:\n')
display(boston_input.info())

# drop neighborhood from the data being considered
boston = boston_input.drop('neighborhood', 1)
print('\n\nGeneral description of the boston DataFrame:\n')

display(boston.info())

print('\nDescriptive statistics of the boston DataFrame:\n')
display(boston.describe())

In [ ]:
boston.hist()

## Correlation Heatmap

In [ ]:
sns.set_theme(style="white")

#correlation matrix
corr =boston.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.subplots_adjust(top=1)
plt.suptitle("Correlation Heatmap for Boston",fontsize= 30)


In [ ]:
sns.scatterplot(boston.ptratio, boston.tax)

In [ ]:
sns.scatterplot(boston.tax, np.log(boston.mv))


In [ ]:
inv_ratio= 1/boston.ptratio
inv_ratio.hist()


In [ ]:
boston.lstat.hist(bins=15)

In [ ]:
log = np.sqrt(boston.lstat)

log.hist()



In [ ]:
boston.rooms.hist()

In [ ]:
np.log(boston.mv).hist()

In [ ]:
logplot = sns.scatterplot(y=np.log(boston.mv), x=boston.lstat)

In [ ]:
sqrtfeatureplot = sns.scatterplot(y=np.log(boston.mv), x= np.sqrt(boston.lstat))

In [ ]:
sqrtfeatureplot = sns.scatterplot(y=boston.mv, x= 1/boston.nox)


# C: Scientist's approach; select correlated features
## C1.) Data Preparation

In [ ]:
boston_trimmed = boston[['mv', 'nox', 'lstat', 'ptratio', 'rooms', 'tax']]

boston_trimmed['sqrt_lstat'] = np.sqrt(boston_trimmed['lstat'])
boston_trimmed['log_MV']= np.log(boston_trimmed['mv'])

boston_trimmed.drop(['lstat', 'mv'], axis=1, inplace=True)


In [ ]:
sns.set_theme(style="white")

#correlation matrix
corr =boston_trimmed.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.subplots_adjust(top=1)
plt.suptitle("Correlation Heatmap for Boston_trimmed",fontsize= 30)

### Here the scientist selects features sqrt_lstat, tax

In [63]:
# here is the data used in the scientist selected features
model_data =boston_trimmed[['log_MV','sqrt_lstat', 'tax']]

In [ ]:
Trimmed_Train, Trimmed_Test = train_test_split(model_data, test_size = 0.2, random_state = RANDOM_SEED)

#separate features from labels

T_Test_y = Trimmed_Test['log_MV'].copy()
T_Test_X = Trimmed_Test.drop('log_MV', axis=1)

T_Train_y = Trimmed_Train['log_MV'].copy()
T_Train_X = Trimmed_Train.drop('log_MV', axis=1)




## C2.) Linear Regression of selected features

In [ ]:
Linear_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', LinearRegression())
        ])

In [ ]:
#fit our model
Linear_pipe.fit(T_Train_X, T_Train_y)

#predict the labels from the train values
Linear_T_train_pred = Linear_pipe.predict(T_Train_X)

#parity plot
fig, T_train = plt.subplots()
T_train.scatter(T_Train_y, Linear_T_train_pred)
T_train.plot([T_Train_y.min(), T_Train_y.max()], [T_Train_y.min(), T_Train_y.max()], 'k--', lw=4)
T_train.set_xlabel('Measured')
T_train.set_ylabel('Predicted')

plt.show()


### Model Summary Stats (Linear Train)

In [ ]:
#regression statistics functions
regression_results(T_Train_y, Linear_T_train_pred)

linear_coefs = np.append(Linear_pipe.named_steps['regressor'].intercept_, Linear_pipe.named_steps['regressor'].coef_)
model_statistics(x_actual=T_Train_X, y_actual=T_Train_y, y_predict=Linear_T_train_pred, coefs=linear_coefs)

### CrossValidation

In [ ]:
#kfold construction
kfold = KFold(n_splits=8, random_state=RANDOM_SEED)

#get RMSE for the folds
cv_results = np.sqrt(-cross_val_score(Linear_pipe, T_Train_X, T_Train_y, cv=kfold, scoring= 'neg_mean_squared_error'))
print(cv_results)
np.mean(cv_results)

### Evaluate on Test

In [ ]:
#fit our model on test data
Linear_pipe.fit(T_Test_X, T_Test_y)

#make predictions of the test labels
Linear_T_test_pred =Linear_pipe.predict(T_Test_X)

#parity plot
fig, Linear_test = plt.subplots()
Linear_test.scatter(T_Test_y, Linear_T_test_pred)
Linear_test.plot([T_Train_y.min(), T_Train_y.max()], [T_Train_y.min(), T_Train_y.max()], 'k--', lw=4)
Linear_test.set_xlabel('Measured')
Linear_test.set_ylabel('Predicted')

plt.show()


### Model Summary Stats (Linear Test)

In [ ]:
#our summary statistic functions
regression_results(T_Test_y, Linear_T_test_pred)

linear_coefs = np.append(Linear_pipe.named_steps['regressor'].intercept_, Linear_pipe.named_steps['regressor'].coef_)
model_statistics(x_actual=T_Test_X, y_actual=T_Test_y, y_predict=Linear_T_test_pred, coefs=linear_coefs)

## C3.) Scientist Approach ElasticNet Regression

In [ ]:
Elastic_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', ElasticNet(alpha=0.005, l1_ratio= 0.5))
        ])

In [ ]:
#fit model with EN on train
Elastic_pipe.fit(T_Train_X, T_Train_y)

#predict our outcomes of train
Elastic_T_train_pred = Elastic_pipe.predict(T_Train_X)

#parity plot
fig, Elastic_train = plt.subplots()
Elastic_train.scatter(T_Train_y, Linear_T_train_pred)
Elastic_train.plot([T_Train_y.min(), T_Train_y.max()], [T_Train_y.min(), T_Train_y.max()], 'k--', lw=4)
Elastic_train.set_xlabel('Measured')
Elastic_train.set_ylabel('Predicted')

plt.show()

### ElasticNet Model Summary Stats (Train)

In [ ]:
# Regression statistics functions again
regression_results(T_Train_y, Elastic_T_train_pred)

EN_coefs = np.append(Elastic_pipe.named_steps['regressor'].intercept_, Elastic_pipe.named_steps['regressor'].coef_)
model_statistics(x_actual=T_Train_X, y_actual=T_Train_y, y_predict=Elastic_T_train_pred, coefs=EN_coefs)

### Cross validation ElasticNet (Train)

In [ ]:
# CrossValidation
cv_results = np.sqrt(-cross_val_score(Elastic_pipe, T_Train_X, T_Train_y, cv=kfold, scoring= 'neg_mean_squared_error'))
print(cv_results)
np.mean(cv_results)

### Evaluate on Test

In [ ]:
#fit the test data using EN
Elastic_pipe.fit(T_Test_X, T_Test_y)

#predict our test labels using EN
Elastic_T_test_pred = Elastic_pipe.predict(T_Test_X)

#pairty plot
fig, Elastic_test = plt.subplots()
Elastic_test.scatter(T_Test_y, Elastic_T_test_pred)
Elastic_test.plot([T_Train_y.min(), T_Train_y.max()], [T_Train_y.min(), T_Train_y.max()], 'k--', lw=4)
Elastic_test.set_xlabel('Measured')
Elastic_test.set_ylabel('Predicted')

plt.show()



### ElasticNet model summary statistics (Test)

In [ ]:
#our regression statistics again
regression_results(T_Test_y, Elastic_T_test_pred)

EN_coefs = np.append(Elastic_pipe.named_steps['regressor'].intercept_, Elastic_pipe.named_steps['regressor'].coef_)
model_statistics(x_actual=T_Test_X, y_actual=T_Test_y, y_predict=Elastic_T_test_pred, coefs=EN_coefs)


# D. Machine Learning Approach
## D1.) Data Prep

In [ ]:
#get our data back in order
boston_new =boston.copy()
boston_new['sqrt_lstat'] = np.sqrt(boston_new['lstat'])
boston_new['log_MV']= np.log(boston_new['mv'])

boston_new.drop(['lstat', 'mv'], axis=1, inplace=True)



In [ ]:
# split our data now that we have all features again
Train, Test = train_test_split(boston_new, test_size = 0.2, random_state = RANDOM_SEED)

#separate features from labels for test
init_Test_Labels = Test['log_MV'].copy()
init_Test_Features = Test.drop('log_MV', axis=1)

#separate features from labels for train
init_y = Train['log_MV'].copy()
init_X = Train.drop('log_MV', axis=1)


## D2.) Lasso Regression on Train

In [ ]:
lasso_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', Lasso(alpha=0.1))
        ])

In [ ]:
#fit our data using lasso
lasso_pipe.fit(init_X, init_y)
lasso_Xtrain_pred =lasso_pipe.predict(init_X)

#pairty plot
fig, Lasso_train = plt.subplots()
Lasso_train.scatter(init_y, lasso_Xtrain_pred)
Lasso_train.plot([init_y.min(), init_y.max()], [init_y.min(), init_y.max()], 'k--', lw=4)
Lasso_train.set_xlabel('Measured')
Lasso_train.set_ylabel('Predicted')

plt.show()



### Lasso Training Model Statistics

In [ ]:
# regression statistics functions
regression_results(init_y, lasso_Xtrain_pred)

lasso_coefs = np.append(lasso_pipe.named_steps['regressor'].intercept_, lasso_pipe.named_steps['regressor'].coef_)
model_statistics(x_actual=init_X, y_actual=init_y, y_predict=lasso_Xtrain_pred, coefs=lasso_coefs)

### Lasso impactful features
    most impactful features (in descending order) are: sqrt_lstat, crim, rooms, ptratio (this last one is pretty small

In [ ]:
# Get our model formula
print ("Lasso model:", pretty_print_coefs(lasso_pipe.named_steps['regressor'].coef_))


In [ ]:
#corresponding features for the model formula
init_X.columns

Cross Validation of the train data using lasso

In [ ]:
cv_results = np.sqrt(-cross_val_score(lasso_pipe, init_X, init_y, cv=kfold, scoring= 'neg_mean_squared_error'))
print(cv_results)
np.mean(cv_results)

### Lasso evaluation on test data

In [ ]:
#fit lasso on test data
lasso_pipe.fit(init_Test_Features, init_Test_Labels)

#predict test labels using lasso
Lasso_test_pred = lasso_pipe.predict(init_Test_Features)

#parity plot
fig, Lasso_test = plt.subplots()
Lasso_test.scatter(init_Test_Labels, Lasso_test_pred)
Lasso_test.plot([init_Test_Labels.min(), init_Test_Labels.max()], [init_Test_Labels.min(), init_Test_Labels.max()], 'k--', lw=4)
Lasso_test.set_xlabel('Measured')
Lasso_test.set_ylabel('Predicted')

plt.show()


### Lasso Model Statistics (test)

In [ ]:
# model statistics functions
lasso_Test_pred = lasso_pipe.predict(init_Test_Features)
regression_results(init_Test_Labels, lasso_Test_pred)

lasso_coefs = np.append(lasso_pipe.named_steps['regressor'].intercept_, lasso_pipe.named_steps['regressor'].coef_)
model_statistics(x_actual=init_Test_Features, y_actual=init_Test_Labels, y_predict=lasso_Test_pred, coefs=lasso_coefs)

# D2.) LinearRegression

## D2a.) Linear Regression on Train (all features)

In [ ]:
Linear_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', LinearRegression())
        ])

In [ ]:
#fit all features with linear regressor
Linear_pipe.fit(init_X, init_y)

#predict labels on all features 
Linear_Xsub_train_pred =Linear_pipe.predict(init_X)

#parity plot
fig, Lin_train = plt.subplots()
Lin_train.scatter(init_y, Linear_Xsub_train_pred)
Lin_train.plot([init_y.min(), init_y.max()], [init_y.min(), init_y.max()], 'k--', lw=4)
Lin_train.set_xlabel('Measured')
Lin_train.set_ylabel('Predicted')

plt.show()

### Linear Regression Statistics (all features)

In [ ]:
# model statistics functions
regression_results(init_y, Linear_Xsub_train_pred)

sub_coefs = np.append(Linear_pipe.named_steps['regressor'].intercept_, Linear_pipe.named_steps['regressor'].coef_)

model_statistics(x_actual= init_X, y_actual= init_y, y_predict=Linear_Xsub_train_pred, coefs=sub_coefs)

## From this we can see that R2 is higher (RMSE also lower) for linear regression. 
    1) However, there are insignificant terms in our model indicated by low t-stats and high p-values in the summary table above.
    2) The regularization of the lasso regressior penalizes the contribution of those coefficients such that it sets them to zero.
    
### Let's use the most prominant features from lasso regression (those with the largest coefficients) for our linear regression

## D2b.) Linear Regression with lasso downselected features

In [ ]:
boston_subset = Train[['sqrt_lstat', 'crim' , 'rooms', 'log_MV']]

X_sub = boston_subset.iloc[:,0:3]
y_sub = boston_subset.iloc[:,-1]

X_sub.columns

In [ ]:
Linear_pipe.fit(X_sub, y_sub)

Linear_Xsub_train_pred =Linear_pipe.predict(X_sub)

fig, train = plt.subplots()
train.scatter(y_sub, Linear_Xsub_train_pred)
#plt.plot(Train_Features, Train_Labels_pred, color='blue', linewidth=3)
train.plot([y_sub.min(), y_sub.max()], [y_sub.min(), y_sub.max()], 'k--', lw=4)
train.set_xlabel('Measured')
train.set_ylabel('Predicted')

plt.show()

### Downselected Linear Regression model statistics

In [ ]:
regression_results(y_sub,Linear_Xsub_train_pred)
sub_coefs = np.append(Linear_pipe.named_steps['regressor'].intercept_, Linear_pipe.named_steps['regressor'].coef_)
model_statistics(x_actual= X_sub, y_actual= y_sub, y_predict=Linear_Xsub_train_pred, coefs=sub_coefs)

In [ ]:
kfold = KFold(n_splits=8, random_state=RANDOM_SEED)

#get RMSE for the folds
cv_results = np.sqrt(-cross_val_score(Linear_pipe, X_sub, y_sub, cv=kfold, scoring= 'neg_mean_squared_error'))
print(cv_results)
np.mean(cv_results)

## implement the identified feature downselect for linear regression on the test data

In [ ]:
Test_subset = Test[['sqrt_lstat', 'crim' , 'rooms','log_MV']]
Test_X_sub = Test_subset.iloc[:,0:3]
Test_y_sub = Test_subset.iloc[:,-1]


In [ ]:
Linear_pipe.fit(Test_X_sub, Test_y_sub)

Linear_Xsub_test_pred =Linear_pipe.predict(Test_X_sub)

fig, Test_ln = plt.subplots()
Test_ln.scatter(Test_y_sub, Linear_Xsub_test_pred)
#plt.plot(Train_Features, Train_Labels_pred, color='blue', linewidth=3)
Test_ln.plot([Test_y_sub.min(), Test_y_sub.max()], [Test_y_sub.min(), Test_y_sub.max()], 'k--', lw=4)
Test_ln.set_xlabel('Measured')
Test_ln.set_ylabel('Predicted')

plt.show()

### Summary Statistis for downselected linear regression (test)

In [ ]:
### Noregression_results(y_sub,Linear_Xsub_train_pred)
regression_results(Test_y_sub,Linear_Xsub_test_pred)
test_sub_coefs = np.append(Linear_pipe.named_steps['regressor'].intercept_, Linear_pipe.named_steps['regressor'].coef_)
model_statistics(x_actual= Test_X_sub, y_actual= Test_y_sub, y_predict=Linear_Xsub_test_pred, coefs=test_sub_coefs)